## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
'eeee'



'eeee'

### Шаг 1. Откройте файл с данными и изучите общую информацию. 


In [1]:
import pandas as pd
data = pd.read_csv('data.csv', delimiter=',')
data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


- Вывод

Перед нами список, состоящий из 12 столбцов и тысяч строк.

В строках встречаются различные типы данных, целые числа, вещественные и класс object. 

В столбце общий трудовой стаж: days employee  встречаются отрицательные значения, чего быть не должно

В столбце education значения забиты в разном регистре, что помешает дальнейшему анализу.

А в столбце purpose сразу бросается в глаза разичная формулировка целей: "приоберетние автомобиля" и "покупка автообил"я - это об одном и том же, поэтому необходимо найти все формлировки и привести их к одной общей.  

Так же с помощью метода .info(), я определила, что есть пропущенные значения в столбце с индексом 1 и 10.


### Шаг 2. Предобработка данных


In [3]:
##Замените названия столбцов (приведите к нижнему регистру);
##Преобразуйте данные в нужные типы. Опишите, в каких столбцах заменили тип данных и почему;
##Обработайте пропуски при необходимости:
##Объясните, почему заполнили пропуски определённым образом или почему не стали это делать;
##Опишите причины, которые могли привести к пропускам;
##Обратите внимание на аббревиатуру 'tbd' в столбцах с рейтингом. Отдельно разберите это значение и опишите, как его обработать;
##Посчитайте суммарные продажи во всех регионах и запишите их в отдельный столбец.

Исследовала столбцы на нахождение данных "с ошибкой"

Так как в data['days_employed'] преобладающее большинство значений записаны со знаком минус, а отрицательного стажа работы быть не должно, нам следует изменить значения на положительные. 


А в столбце data['education'] привести все значения к одному регистру.

In [3]:
data['education'].value_counts()
data['education'].str.lower()

0         высшее
1        среднее
2        среднее
3        среднее
4        среднее
          ...   
21520    среднее
21521    среднее
21522    среднее
21523    среднее
21524    среднее
Name: education, Length: 21525, dtype: object

In [4]:
data['days_employed'].value_counts()

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64

In [5]:
a = 0
for i in data['days_employed']:
    if i < 0:
        a += 1
    

In [6]:
a

15906

In [7]:
lenght_not_null_days_emp = len(data) - len(data[data['days_employed'].isna()])
percent_plus_days_emp = round(a / lenght_not_null_days_emp * 100, 2)
percent_plus_days_emp

82.2

Вывод: мы посчитали количество отрицательных значений с помощью цикла. 
Сохранили значение в переменной а, затем посчитали отношение количества отрицательных значений к количеству всех фреймов в сериусе.
Затем посчитали и сохранили значениче колличества заполненных фреймов, после того как вычли количество пропущенных значений из количества всех фремов
И нашли процентное отношение отрицательных значений к количеству заполненных фреймов в сериусе. 
Так как процент достаточно большой, я задумалась, может ли это быть ошибкой (человеческий фактор) или значения не верны и стоит их просто удалить. 

Кредитный скоринг - система оценки, основанная на численных статистических методах. Стаж работы важный критерий в оценке кредитоспособности лица, удалить эти значения без разрешения банка я не могу, потому воспользуюсь методом замены отрицательных значений на положительные


In [8]:
len(data.query('days_employed < 0 '))

15906

Замена циклу

In [9]:
data[data['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [10]:
data['dob_years'].value_counts()
len(data.query('dob_years < 18 '))
#заемщик кредита в банке не может быть младше 18 лет, здесь ошибка

101

In [11]:
round(len(data.query('dob_years < 18 ')) / len(data) * 100, 2)


0.47

In [12]:
data = data[data['dob_years'] >= 18]
len(data.query('dob_years < 18'))

0

В процентном соотношении получается меньше 1% от всех значений датафрейма, было принято решение удалить данные

In [13]:
data['children'].value_counts()
#здесь следует заменить значения -1 и 20 на 1 и 20 соответственно, 
#скорее всего это опечатки

 0     14080
 1      4802
 2      2042
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

### Обработка пропусков

In [14]:
len(data[data['days_employed'].isna()])

2164

In [15]:
data['days_employed'] = abs(data['days_employed'])


In [16]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь стоит решить, что делать с пропущенными значениями, которых 2174! Найдем их процентное соотноение

In [17]:
round(len(data[data['days_employed'].isna()]) / len(data) * 100, 2)

10.1

10 % это немного, я скорее отнесу это к ошибке технического характера и заменю данные на средний стаж работы. Так как удаление данных приведет к их потере и, возможно, к ошибкам в выводах


In [15]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean())

In [ ]:
# Решение главного аналитика Алексея Бельтюкова.
# Подойдем к заполнению пропусков более искуссно и заменим их согласно категорям.

In [19]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [ ]:
# 1 Можно использовть как категории персентили dob_years
# 2 Также можно использовать education
# 3 Также можно использовать income_type

In [20]:
data.dob_years.max()

75

In [21]:
data.dob_years.min()

19

In [29]:
years_perc_df = pd.DataFrame()
years_perc_df['years'] = list(range(19,76))

In [30]:
print(years_perc_df.years.quantile(0.25))
print(years_perc_df.years.quantile(0.5))
print(years_perc_df.years.quantile(0.75))

33.0
47.0
61.0


In [32]:
import numpy as np

In [33]:
data[(data['dob_years'] > 0)&(data['dob_years'] <= 33)].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
15,1,972.364419,26,среднее,1,женат / замужем,0,F,сотрудник,0,116820.904450,строительство собственной недвижимости
17,0,2369.999720,33,высшее,0,гражданский брак,1,M,сотрудник,0,90410.586745,строительство недвижимости


In [35]:
np.NaN

nan

In [55]:
for row in range(len(data)):
    if data.iloc[row]['days_employed'] == np.NaN and data.iloc[row]['dob_years'] >= 0 and data.iloc[row]['dob_years'] <= 33:
        data.iloc[row]['days_employed'] = data[(data['dob_years'] >= 0)&(data['dob_years'] <= 33)]['days_employed'].mean()
    elif data.iloc[row]['days_employed'] == np.NaN and data.iloc[row]['dob_years'] > 33  and data.iloc[row]['dob_years'] <= 47:
        data.iloc[row]['days_employed'] = data[(data['dob_years'] > 33)&(data['dob_years'] <= 47)]['days_employed'].mean()
    elif data.iloc[row]['days_employed'] == np.NaN and data.iloc[row]['dob_years'] > 47 and data.iloc[row]['dob_years'] <= 61:
        data.iloc[row]['days_employed'] = data[(data['dob_years'] > 47)&(data['dob_years'] <= 61)]['days_employed'].mean()
    elif data.iloc[row]['days_employed'] == np.NaN and data.iloc[row]['dob_years'] > 61:
        data.iloc[row]['days_employed'] = data[(data['dob_years'] > 61)]['days_employed'].mean()
        

Мы заменили пропущенные значения в сериусе data['days_employed']. Так же пропущенные значения встречаются в сериусе total_income в таком же количестве у тех же людей, скорее это тоже ошибка технического характера, так как сферв занятости семейный статус, возраст клиентов разный и нет видимых причин, почему пропуски могли бы быть оставлены специально. Проведем с ним такую же операцию. И сделаем проверку таблицы методом isna() на наличие пропусков

In [16]:
data['total_income'] = data['total_income'].fillna(data['total_income'].mean())

In [17]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [20]:
data['gender'].value_counts()
#мы обнаружили один пропуск в граве "пол", так как он всего 1, его можно удалить

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Пропущенных значений больше нет. Мы заполнили их средним значением по соответвующему столбцу, так как удаление 1/10 данных может привести к ошибке в выводе.


## Замена типа данных

После обработки пропусков нам следует проверить другие столбцы на наличие артефактов. 

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [22]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [23]:
data['total_income'].value_counts()

167422.302208    2174
104381.857170       1
184315.121979       1
182036.676828       1
122421.963500       1
                 ... 
133299.194693       1
115080.782380       1
84896.781597        1
153838.839212       1
150014.128510       1
Name: total_income, Length: 19352, dtype: int64

###Лемматизация

In [24]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Замена типа данных
### Обработка дубликатов
### Лемматизация
### Категоризация данных
### Шаг 3. Ответьте на вопросы
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?x
- Как разные цели кредита влияют на его возврат в срок?
### Шаг 4. Общий вывод


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.
- [x ]  открыт файл;
- [ x]  файл изучен;
- [ x]  определены пропущенные значения;
- [ x]  заполнены пропущенные значения;
- [ x]  есть пояснение какие пропущенные значения обнаружены;
- [ x]  описаны возможные причины появления пропусков в данных;
- [ x]  объяснено по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.